In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gzip
import os
import h5py

In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 98 kB 1.9 MB/s 


In [ ]:
import keras_tuner as kt

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive/Group 4/

/content/drive/.shortcut-targets-by-id/1-A7MSZ6V8eP_xZaDJ_j7yN9ddVL8-jgm/Group 4


In [ ]:
def model_builder(hp):

    model = keras.models.Sequential()

    hp_units_1 = hp.Int('units_1', min_value=20, max_value=30, step=1)

    model.add(layers.Conv2D(hp_units_1, kernel_size=(5,20), 
                            activation='relu', 
                            input_shape=(64,256,1), 
                            bias_regularizer=regularizers.l1(1e-5),
                            activity_regularizer=regularizers.l1(1e-5),
                            kernel_regularizer=regularizers.l1(1e-5)))
    model.add(layers.MaxPooling2D(pool_size=(3,12)))

    hp_units_2 = hp.Int('units_2', min_value=10, max_value=20, step=1)

    model.add(layers.Flatten())
    model.add(layers.Dense(hp_units_2, activation='relu',
              bias_regularizer=regularizers.l1(1e-5),
              activity_regularizer=regularizers.l1(1e-5),
              kernel_regularizer=regularizers.l1(1e-5)))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-4),
                  loss=keras.losses.BinaryCrossentropy(),
                  metrics=tf.metrics.BinaryAccuracy())

    return model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [ ]:
root = os.getcwd()
save_path = root + '/Models/S1/cnn_cv_checkpoints'

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_binary_accuracy',
                     max_epochs=5,
                     factor=3, 
                     directory=save_path,
                     project_name='cnn_cv')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_binary_loss', patience=5)

Function to list patient names

In [ ]:
def list_patient_names(root):

    patient_list = sorted([item for item in os.listdir(root) 
                           if os.path.isdir(os.path.join(root, item))],
                           key = lambda x: x[4:])
    
    return patient_list

Function to list trial files

In [ ]:
def list_trial_files(root, patient_name):
    
    patient_folder = root + patient_name + "/"
    
    trial_names = sorted([f for f in os.listdir(patient_folder) 
                          if os.path.isfile(os.path.join(patient_folder, f))])
    
    return trial_names

Function to read in patient name, patient type, trial number, stimulus type, sensor name

In [ ]:
def parse_dat(root, patient_name, trial_file):
    
    txtfile = root + patient_name + "/" + trial_file
    
    # Read in the entire data file
    lines = []

    with gzip.open(txtfile,'rt') as f:
        for line in f:
            lines.append(line)
            
    # Get the patient name
    patient_name = lines[0].split(' ')[1][:-4]
    
    # Get patient type (alcoholic or control)
    patient_type = patient_name[3]
    
    # Get trial number
    trial_num = trial_file[-6:-3]
    
    # Get stimulus type
    stim_type = lines[3].split(' ')[1] + ' ' + lines[3].split(' ')[2]
    
    # Get sensor names
    readings = lines[4:]
    headers = [r for r in readings if r[0]=='#']
    sensor_names = [h.split(' ')[1] for h in headers]
        
    # Get sensor readings for all 64 detectors for all 256 timestamps
    data = [r for r in readings if r[0]!='#']
    
    data_mat = np.zeros((64, 256))
    
    for i in range(64):
        for j in range(256):
            data_mat[i, j] = float(data[i*256+j].split(' ')[3][:-1])
            
    return patient_name, patient_type, trial_num, stim_type, sensor_names, data_mat

Load in all the data

In [ ]:
data_path = root + '/eeg_full/'

In [ ]:
df = pd.read_csv(data_path+'S1_patient_summary.csv')
hf = h5py.File(data_path+'S1.h5', 'r')

In [ ]:
df['Type'] = df['Type'].replace({'c':0, 'a':1})

In [ ]:
dat_tensor = np.zeros((len(df), 64, 256))

In [ ]:
dat_targets = np.zeros(len(df))

In [ ]:
for i in range(len(df)):
    
    pat = df.iloc[i]['Patient']
    tri = '%03d'%df.iloc[i]['Trial']
    
    fname = pat+'.rd.'+tri
    
    dat = np.array(hf.get(fname))
    
    dat_tensor[i] = dat
    dat_targets[i] = df.iloc[i]['Type']

Split training and testing data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_tensor, test_tensor, train_target, test_target = train_test_split(
                                            dat_tensor, dat_targets, 
                                            test_size=0.2, random_state=0)

Construct a neural network

In [ ]:
tuner.search(train_tensor, train_target, epochs=5, 
             validation_split=0.2, callbacks=[stop_early])

Trial 10 Complete [00h 00m 14s]
val_binary_accuracy: 0.6932725310325623

Best val_binary_accuracy So Far: 0.7206385135650635
Total elapsed time: 00h 01m 50s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
hyperparams = np.array([best_hps.get('units_1'),
                        best_hps.get('units_2')])

In [ ]:
hyperparams

array([23, 19])

In [ ]:
hyper_path = root + '/Models/S1/cnn_hyperparams.npy'
np.save(hyper_path, hyperparams)